In [ ]:
from collections import Counter
import sys

from sympy.utilities.iterables import variations
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import numpy as np
import seaborn as sns


import lieb_liniger_state as lls
from visualize import *
import rho_form_factor as rff
from sum_rule import *
import utils

%config InlineBackend.figure_format = 'svg'
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
c = 1
L = N = 5
Imax = 12
N_world = 2 * Imax + 1

rstate = lls.lieb_liniger_state(c, L, N)
rstate.calculate_all()

In [ ]:
perms = set(sorted(tuple(sorted(x)) for x in variations(range(-Imax, Imax + 1), N)))

In [ ]:
k = Counter(sorted(sum(x) for x in perms))
print("# of states:", sum([k[z] for z in k]))

In [ ]:
dsf_data = {}
for i, a in enumerate(perms):
    lstate = lls.lieb_liniger_state(c, L, N, list(a))
    lstate.calculate_all()
    lstate.ff = rff.rho_form_factor(lstate, rstate)
    if lstate.integer_momentum in dsf_data.keys():
        dsf_data[lstate.integer_momentum].append(lstate)
    else:
        dsf_data[lstate.integer_momentum] = [lstate]
    sys.stdout.write(f"{str(i)}\r")
    sys.stdout.flush()

In [ ]:
visualize_no_of_states_per_slice(dsf_data)

In [ ]:
compute_average_sumrule(dsf_data, rstate.energy, L, N, Imax, N_world, print_all=True)

In [ ]:
visualize_sumrule_per_contributing_state(dsf_data, rstate.energy, L, N, xlim=(-Imax-1, Imax+1), save=True)

In [ ]:
# Find the form factors for the states and the assiocated number of particle hole pairs
no_ex_data  = {}
form_facts = {}
sum_rule_contribs = {}
for _, states in sorted(dsf_data.items()):
    for state in states:
        no = utils.no_of_particle_hole_pairs(
            utils.map_to_entire_space(state.Is, Imax), 
            utils.map_to_entire_space(rstate.Is, Imax), N)
        if no in no_ex_data.keys():
            no_ex_data[no].append(state)
        else:
            no_ex_data[no] = [state]
        if no in form_facts.keys():
            form_facts[no].append(state.ff)
        else:
            form_facts[no] = [state.ff]
        if state.integer_momentum != 0:
            if no in sum_rule_contribs.keys():
                sum_rule_contribs[no].append(left_side([state], rstate.energy) / right_side(state.integer_momentum, L, N))
            else:
                sum_rule_contribs[no] = [left_side([state], rstate.energy) / right_side(state.integer_momentum, L, N)]

In [ ]:
fig, ax = plt.subplots()
ax.bar(range(0, len(no_ex_data.keys())), [np.mean(np.abs(np.array(k[1]))**2) for k in sorted(form_facts.items())])
ax.set_xlabel("Number of particle-hole pairs")
ax.set_ylabel("Mean form factor squared")
ax.set_yscale('log')
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
sns.despine()
fig.savefig("ave_form_factor_ph_pairs.pdf", bbox='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.bar(range(0, len(no_ex_data.keys())), [len(no_ex_data[z]) for z in range(0, len(no_ex_data.keys()))])
ax.set_xlabel("Number of particle-hole pairs")
ax.set_ylabel("Number of states")
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
sns.despine()
fig.savefig("no_states_ph_pairs.pdf", bbox='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.bar(range(0, len(no_ex_data.keys())), [0] + [np.mean(np.array(k[1])) for k in sorted(sum_rule_contribs.items())])
ax.set_xlabel("Number of particle-hole pairs")
ax.set_ylabel("Mean sumrule saturation per state")
ax.set_yscale('log')
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
sns.despine()
fig.savefig("ave_sum_rule_saturation_ph_pairs.pdf", bbox='tight')
plt.show()